In [3]:
import numpy as np

class Route:
    def __init__(self, longueur, pheromone, premiere_ville, seconde_ville):
        self.longueur = longueur
        self.pheromone = pheromone
        self.premiere_ville = premiere_ville
        self.seconde_ville = seconde_ville
    
    def evaporer_Pheromone(self, taux):
        self.pheromone *= (1 - taux)

    def __str__(self):
        return "Route de {} à {} de longueur {}".format(self.premiere_ville, self.seconde_ville, self.longueur)

class Ville:
    def __init__(self, nom, position):
        self.nom = nom
        self.position = position
    def __str__(self):
        return self.nom

class Ant:
    def __init__(self, alpha: float, beta: float, gamma: float, porte_nourriture: bool, position):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.porte_nourriture = porte_nourriture
        self.memoire = []
        self.position = position
        self.pas = 0
    
    def choix_arete(self, villes_voisines, routes):
        ville = None
        coeff = 0.5
        q0 = 2
        q = np.random.random()
        VILLE = []
        score=[]
        if q < q0:
            for k in villes_voisines:
                if k not in self.memoire:
                    print(f"Les villes voisines {k}")
                    for route in routes:
                        if k in [route.premiere_ville, route.seconde_ville]:
                            if self.position in [route.premiere_ville, route.seconde_ville]:
                                if self.position == route.premiere_ville:
                                    taux = route.pheromone
                                    heuristique = 1 / route.longueur
                                    VILLE.append(route.seconde_ville)
                                else:
                                    taux = route.pheromone
                                    heuristique = 1 / route.longueur
                                    VILLE.append(route.seconde_ville)
                                score.append(taux * (heuristique ** coeff))
            ville = VILLE[np.argmax(score)]
        else:
            a=0
            l=[]
            while ville is None and a < len(villes_voisines):
                k = np.random.choice(villes_voisines)
                if k not in self.memoire:
                    ville = k
                if k not in l:
                    l.append(k)
                    a += 1
        if ville is None:
            pass
        else:
            self.memoire.append(ville)
            self.pas = 0
            for route in routes:
                if ville in [route.premiere_ville, route.seconde_ville]:
                    if self.position in [route.premiere_ville, route.seconde_ville]:
                        self.position = route
        print(f"Je choisis cela : {self.position}")
    def prendre_nourriture(self):
        if not self.porte_nourriture:
            self.porte_nourriture = True
    
    def laisser_nourriture(self):
        if self.porte_nourriture:
            self.porte_nourriture = False
    
    def deposer_pheromone(self):
        print('Oui')
        PLt = self.position.pheromone
        PLt1 = PLt + abs(self.alpha * np.sin(self.beta * PLt + self.gamma))
        self.position.pheromone = PLt1 
    
    def marcher(self):
        print(f"Mon pas : {self.pas}")
        self.pas +=1
        if self.pas == self.position.longueur :
            if self.position.premiere_ville in self.memoire:
                self.position = self.position.seconde_ville
            else:
                self.position = self.position.premiere_ville
        else :
            self.deposer_pheromone()

class Civilisation:
    def __init__(self, src_nourriture, nid, routes, villes, fourmis, selectionNaturelle):
        self.src_nourriture = src_nourriture  # La ville source de nouriture de la Civilisation
        self.nid = nid  # Le nid de la Civilisation
        self.routes = routes  # toutes les routes de l’Environement
        self.villes = villes  # toutes les villes de l’Environement
        self.fourmis = fourmis  # toutes les fourmis dans l’Environement
        self.selectionNaturelle = selectionNaturelle  # les tours restants avant la prochaine sélection (pour l’algorithme génétique)
        self.iteration = 0  # le nombre de pas effectués par les fourmis

    def tourSuivant(self):# Effectue un tour de la simulation
        print(f"Je me situe en {self.fourmis[0].position}")
        if self.selectionNaturelle > self.iteration:
            for fourmi in self.fourmis:
                villes_voisines = self.villes_voisines(fourmi.position)  # Remplace '__' par 'self.'
                if fourmi.position == self.src_nourriture:
                    fourmi.memoire = [self.src_nourriture]
                    fourmi.prendre_nourriture()
                    fourmi.choix_arete(villes_voisines, self.routes)
                elif fourmi.position == self.nid:
                    fourmi.memoire = [self.nid]
                    fourmi.laisser_nourriture()
                    fourmi.choix_arete(villes_voisines, self.routes)  # Appelle la méthode 'choix_arete'
                elif fourmi.position in self.villes:
                    if not villes_voisines:
                        continue
                    fourmi.choix_arete(villes_voisines, self.routes)  # Appelle la méthode 'choix_arete'
                elif fourmi.position in self.routes:
                    fourmi.marcher()
            self.iteration += 1
            for route in self.routes:
                route.evaporer_Pheromone(0.1)
        else:
            print('Fin de la simulation !')

    def villes_voisines(self, ville):
        return [v for v in self.villes if v != ville and \
                (ville, v) in [(r.premiere_ville, r.seconde_ville) for r in self.routes] or \
                (v, ville) in [(r.premiere_ville, r.seconde_ville) for r in self.routes]]



In [7]:
# Créer une ville avec nom 'A' et position (0,0)
ville_A = Ville('A', (1,1))
ville_B = Ville('B', (1,0))
ville_C = Ville('C', (2,2))
ville_D = Ville('D', (2,1))
ville_E = Ville('E', (2,-1))

nid = Ville('nid', (0,0))
source = Ville('source', (3,0))

villes = [ville_A, ville_B, ville_C, ville_D, ville_E, nid, source]


route_AB = Route(2,1, ville_A, ville_B)
routeDepartA = Route(3, 1, nid, ville_A)
routeDepartB = Route(5, 1, nid, ville_B)
route_AC = Route(6, 1, ville_A, ville_C)
route_AD = Route(4, 1, ville_A, ville_D)
route_BD = Route(1, 1, ville_B, ville_D)
route_BE = Route(2, 1, ville_B, ville_E)
route_CArrivee = Route(6, 1, ville_C, source)
route_DArrivee = Route(7, 1, ville_D, source)
route_EArrivee = Route(4, 1, ville_E, source)

routes = [route_AB, routeDepartA, routeDepartB, route_AC, route_AD, route_BD, route_BE, route_CArrivee, route_DArrivee, route_EArrivee]

fourmis = []
for i in range(1) :
    alpha = np.random.randint(1, 10)
    beta = np.random.randint(1, 10)
    gamma = np.random.randint(1, 10)
    fourmis.append(Ant(alpha, beta, gamma, False, nid))

N=100

civilisation = Civilisation(source, nid, routes, villes, fourmis, N)
for i in range(N):
    civilisation.tourSuivant()


Itération 1
Je me situe en nid
Les villes voisines A
Les villes voisines B
Je choisis cela : Route de nid à A de longueur 3
Taux de pheromone : 0.9
Itération 11
Je me situe en Route de nid à A de longueur 3
Mon pas : 0
Oui
Taux de pheromone : 0.81
Itération 21
Je me situe en Route de nid à A de longueur 3
Mon pas : 1
Oui
Taux de pheromone : 0.7290000000000001
Itération 31
Je me situe en Route de nid à A de longueur 3
Mon pas : 2
Taux de pheromone : 0.6561000000000001
Itération 41
Je me situe en A
Les villes voisines B
Les villes voisines C
Les villes voisines D
Je choisis cela : Route de A à B de longueur 2
Taux de pheromone : 0.5904900000000002
Itération 51
Je me situe en Route de A à B de longueur 2
Mon pas : 0
Oui
Taux de pheromone : 1.0399319025364053
Itération 61
Je me situe en Route de A à B de longueur 2
Mon pas : 1
Taux de pheromone : 0.9359387122827648
Itération 71
Je me situe en B
Les villes voisines D
Les villes voisines E
Je choisis cela : Route de B à D de longueur 1
Taux 

In [1]:
import tkinter as tk

class Application(tk.Frame):
    def __init__(self, master=None, city_coords=None, ant_coords=None):
        super().__init__(master)
        self.master = master
        self.city_coords = city_coords
        self.ant_coords = ant_coords
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        # Créer un canvas
        self.canvas = tk.Canvas(self, width=500, height=500)
        self.canvas.pack()

        # Tracer la ville
        if self.city_coords:
            for i in self.city_coords:
                print(i)
                self.canvas.create_oval(i[0]-10, i[1]-10, i[0]+10, i[1]+10, fill="red")

        # Tracer la fourmi
        if self.ant_coords:
            for i in self.ant_coords:
                self.canvas.create_oval(i[0]-5, i[1]-5, i[0]+5, i[1]+5, fill="black")

        # Ajouter un bouton en haut au centre
        self.button = tk.Button(self, text="Lancer la fonction", command=self.my_function)
        self.button.pack(side=tk.TOP, pady=10)

    def my_function(self):
        # Fonction à exécuter lorsque l'utilisateur clique sur le bouton
        print("Fonction exécutée !")

# Exemple d'utilisation
if __name__ == "__main__":
    city_coords = [(100, 100), (300, 300)]
    ant_coords = [(200, 200), (400, 400)]

    root = tk.Tk()
    app = Application(master=root, city_coords=city_coords, ant_coords=ant_coords)
    app.mainloop()

(100, 100)
(300, 300)
Fonction exécutée !
Fonction exécutée !
Fonction exécutée !
